In [1]:
!pip install dlib

In [2]:
import os
import dlib
import cv2
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Path to the folders containing real and fake images
real_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
fake_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [ ]:
# Load and preprocess images
def load_and_preprocess_images(folder, max_images=6500):
    images = []
    count = 0
    for filename in os.listdir(folder):
        if count >= max_images:
            break
        print(count)
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            # img = dlib.load_rgb_image(img_path)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64,64))
            images.append(np.array(img))
            count += 1
    return images

In [ ]:
# Load the face pose estimator from DLIB
# face_detector = dlib.shape_predictor("/content/gdrive/MyDrive/shape_predictor_68_face_landmarks.dat")

face_detector = dlib.get_frontal_face_detector()
# pose_predictor = dlib.shape_predictor("~/Documents/PES1UG20CS646_HitheshDN/capstone/summer/code/shape_predictor_68_face_landmarks.dat")
predictor = dlib.shape_predictor("/content/gdrive/MyDrive/shape_predictor_68_face_landmarks.dat")

In [ ]:
# Load and preprocess real and fake images
real_images = np.array(load_and_preprocess_images(real_folder_path))


In [ ]:
fake_images = np.array(load_and_preprocess_images(fake_folder_path))

In [ ]:
#not needed
# fake_images = fake_images[:9000]

In [ ]:
# print(Flatten()(real_images[0]))
# print(real_images[0].flatten())
# print(real_images)
# print(fake_images)


In [ ]:
# image_height, image_width, num_channels = real_images[0].shape
# print(real_images.shape)
# print(fake_images.shape)

# print(real_images[0].shape)
# print(real_images[1].shape)
# print(fake_images[0].shape)
# print(fake_images[1].shape)



In [ ]:
# Combine real and fake images and create labels
all_images = np.concatenate((real_images, fake_images), axis=0)
labels = np.concatenate((np.ones(len(real_images)), np.zeros(len(fake_images))), axis=0)

In [ ]:
# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(all_images, labels, test_size=0.3, random_state=42)

# print(train_images)
# print(val_images)
# print(train_labels)
# print(val_labels)

In [3]:

# def extract_face_pose_features(images):
#     features = []
#     for img in images:
#         face_landmarks = face_pose_estimator(img, dlib.rectangle(0, 0, img.shape[1], img.shape[0]))
#         pose_feature = np.array([(point.x, point.y) for point in face_landmarks.parts()])
#         print(pose_feature)
#         features.append(pose_feature)
#     return features

def find_rotation_matrix(landmarks):
    # Convert landmarks to numpy array for easier indexing
    landmarks_array = np.array([[p.x, p.y] for p in landmarks.parts()])

    # Calculate the center of the eyes
    left_eye_center = landmarks_array[36:42].mean(axis=0)
    right_eye_center = landmarks_array[42:48].mean(axis=0)

    # Calculate the angle of rotation (yaw) using the eye centers
    dy = right_eye_center[1] - left_eye_center[1]
    dx = right_eye_center[0] - left_eye_center[0]
    angle = math.atan2(dy, dx) * 180.0 / math.pi

    #calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(tuple(left_eye_center), angle, 1.0)
    rotation_matrix = np.vstack((rotation_matrix, np.array([0, 0, 1])))
    return rotation_matrix

def find_translation_vector(image,landmarks):
    landmarks_array = np.array([[p.x, p.y] for p in landmarks.parts()])

    # to find center of face
    face_center = landmarks_array.mean(axis=0)

    # Calculate the translation vector as the displacement from the center
    translation_vector = face_center - np.array([image.shape[1] / 2, image.shape[0] / 2])

    return translation_vector


def face_pose_estimator(image):
    features = []
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_image)
    if len(faces) > 0:
        face = faces[0]
        landmarks = predictor(gray_image, face)

        # Calculate yaw, pitch, roll angles using rotation matrix
        # You can modify the code from the previous response to extract these angles
        # rotation_matrix = dlib.get_face_chips(image, [landmarks])[0].pose_rot_matrix
        rotation_matrix = find_rotation_matrix(landmarks)
        # translation_vector = dlib.get_face_chip(image, landmarks).pose_T
        translation_vector = find_translation_vector(image,landmarks)
        # print(rotation_matrix)
        # print(translation_vector)

        yaw = math.degrees(math.atan2(rotation_matrix[1, 0], rotation_matrix[0, 0]))
        pitch = math.degrees(math.atan2(-rotation_matrix[2, 0], math.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2)))
        roll = math.degrees(math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2]))

        # Append angles to the features list
        features=[yaw, pitch, roll]
    else:
        # Append some default values if no face detected
        features=[0, 0, 0]
    return features

In [ ]:
final_training_set = []
final_validating_set = []
for image in train_images:
  img_feature = face_pose_estimator(image)
  temp = []
  temp.extend(image.flatten())
  temp.extend(img_feature)
  final_training_set.append(temp)
for image in val_images:
  img_feature = face_pose_estimator(image)
  temp = []
  temp.extend(image.flatten())
  temp.extend(img_feature)
  final_validating_set.append(temp)

final_training_set = np.array(final_training_set)
final_validating_set = np.array(final_validating_set)

# print(face_pose_estimator(train_images[0]))
# print("----------------------------------")
# print(face_pose_estimator(train_images[1]))
# print("----------------------------------")

# print(face_pose_estimator(train_images[2]))
# print("----------------------------------")

# print(face_pose_estimator(fake_images[1]))

In [4]:
# Define your neural network architecture
def build_model(input_shape=(12291,)):
    input_layer = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [5]:
# Define model architecture
# feature_dim = train_features_flatten.shape[1]
model = build_model()

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10

model.fit(final_training_set, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(final_validating_set, val_labels))

Epoch 1/10
223/223 [==============================] - 9s 17ms/step - loss: 0.8382 - accuracy: 0.5523 - val_loss: 0.6427 - val_accuracy: 0.6395
Epoch 2/10
223/223 [==============================] - 3s 13ms/step - loss: 0.7158 - accuracy: 0.6005 - val_loss: 0.6374 - val_accuracy: 0.6461
Epoch 3/10
223/223 [==============================] - 3s 11ms/step - loss: 0.6744 - accuracy: 0.6200 - val_loss: 0.6290 - val_accuracy: 0.6451
Epoch 4/10
223/223 [==============================] - 3s 11ms/step - loss: 0.6485 - accuracy: 0.6426 - val_loss: 0.6167 - val_accuracy: 0.6553
Epoch 5/10
223/223 [==============================] - 2s 11ms/step - loss: 0.6310 - accuracy: 0.6591 - val_loss: 0.5980 - val_accuracy: 0.6773
Epoch 6/10
223/223 [==============================] - 4s 16ms/step - loss: 0.6180 - accuracy: 0.6744 - val_loss: 0.5820 - val_accuracy: 0.7193
Epoch 7/10
223/223 [==============================] - 3s 14ms/step - loss: 0.5979 - accuracy: 0.6928 - val_loss: 0.5691 - val_accuracy: 0.7272

In [ ]:
batch_size = 64
epochs = 10

model.fit(final_training_set, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(final_validating_set, val_labels))

Epoch 1/10
112/112 [==============================] - 2s 17ms/step - loss: 0.5759 - accuracy: 0.7146 - val_loss: 0.5962 - val_accuracy: 0.7111
Epoch 2/10
112/112 [==============================] - 2s 14ms/step - loss: 0.5659 - accuracy: 0.7203 - val_loss: 0.5954 - val_accuracy: 0.7058
Epoch 3/10
112/112 [==============================] - 2s 15ms/step - loss: 0.5601 - accuracy: 0.7224 - val_loss: 0.5653 - val_accuracy: 0.7209
Epoch 4/10
112/112 [==============================] - 2s 19ms/step - loss: 0.5588 - accuracy: 0.7303 - val_loss: 0.5427 - val_accuracy: 0.7469
Epoch 5/10
112/112 [==============================] - 2s 17ms/step - loss: 0.5588 - accuracy: 0.7294 - val_loss: 0.5847 - val_accuracy: 0.7255
Epoch 6/10
112/112 [==============================] - 2s 13ms/step - loss: 0.5566 - accuracy: 0.7372 - val_loss: 0.6108 - val_accuracy: 0.6999
Epoch 7/10
112/112 [==============================] - 2s 14ms/step - loss: 0.5585 - accuracy: 0.7310 - val_loss: 0.5546 - val_accuracy: 0.7423

In [ ]:
# Train the model
batch_size = 32
epochs = 10

model.fit(final_training_set, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(final_validating_set, val_labels))

Epoch 1/10
254/254 [==============================] - 10s 13ms/step - loss: 0.9021 - accuracy: 0.5654 - val_loss: 0.6511 - val_accuracy: 0.6445
Epoch 2/10
254/254 [==============================] - 3s 12ms/step - loss: 0.7286 - accuracy: 0.6080 - val_loss: 0.6245 - val_accuracy: 0.6534
Epoch 3/10
254/254 [==============================] - 3s 12ms/step - loss: 0.6797 - accuracy: 0.6250 - val_loss: 0.6111 - val_accuracy: 0.6588
Epoch 4/10
254/254 [==============================] - 4s 14ms/step - loss: 0.6429 - accuracy: 0.6495 - val_loss: 0.5935 - val_accuracy: 0.6849
Epoch 5/10
254/254 [==============================] - 3s 13ms/step - loss: 0.6106 - accuracy: 0.6863 - val_loss: 0.6336 - val_accuracy: 0.6573
Epoch 6/10
254/254 [==============================] - 3s 12ms/step - loss: 0.5939 - accuracy: 0.6990 - val_loss: 0.5667 - val_accuracy: 0.7277
Epoch 7/10
254/254 [==============================] - 3s 11ms/step - loss: 0.5802 - accuracy: 0.7103 - val_loss: 0.5558 - val_accuracy: 0.741

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(final_validating_set, val_labels)
print("Test accuracy:", test_accuracy)

96/96 [==============================] - 0s 4ms/step - loss: 0.5355 - accuracy: 0.7482
Test accuracy: 0.7481943368911743


In [10]:
%pip install torchviz
import torch
import torchviz

# pass your input to the model (in your case image of what ever size the input layer takes)
dot = torchviz.make_dot(model(torch.randn(1, 32, 32)))

dot.render("model_architecture.png", format="png")

AttributeError: ignored